In [50]:
from process_image import generate_image
from upload_to_ipfs import upload_to_ipfs
from update_metadata import update_metadata

## Infinity loop to process new txns

In [ ]:
from web3 import Web3
import os
from solcx import compile_source, install_solc
import solcx
from solcx import compile_source
import math
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
chainId=1337
exploit_address = ''#TODO: Place the deployed address from deploy.ipynb here

solcx.install_solc('0.5.17')
gas_limit = 8000000
chainId=9090

with open("ShibaPayment.sol") as f:
    contract_source = f.read()
compiled_sol = compile_source(contract_source, solc_version="0.5.17")
contract_interface = compiled_sol[f'<stdin>:ShibaPayment']
abi = contract_interface['abi']
contract = w3.eth.contract(abi=contract_interface['abi'],
                           bytecode=contract_interface['bin'])


deployed_contract = w3.eth.contract(address=exploit_address, abi=contract_interface['abi'])


In [ ]:
private_key = "" #TODO: Place owner's private key here

account = w3.eth.account.from_key(private_key)

In [ ]:
from time import sleep
def process_user_payments(user_address):
    # Get total payments for this user
    payment_count = deployed_contract.functions.userPaymentCount(user_address).call()
    
    
    for index in range(payment_count):
        try:
            # Get payment details for each index
            payment = deployed_contract.functions.userPayments(user_address, index).call()
            sender, fee, tokenId, status, url = payment
            if status == 0:
                print(f"\nProcessing {payment_count} payments for user {user_address}")
                print()
                print(f"\nPayment {index + 1}:")
                print(f"Sender: {sender}")
                print(f"Fee: {fee}")
                print(f"TokenId: {tokenId.hex()}")
                print(f"status: {status}")
                print(f"url: {url}")
                
                # Here you can add your IPFS upload and metadata update logic
                image_path = generate_image('', tokenId.hex(), sender)
                ipfs_hash = upload_to_ipfs(image_path)
                metadata_ipfs_hash = update_metadata(tokenId.hex(), ipfs_hash, sender, '', fee)
                image_url = f"https://indigo-rapid-swan-980.mypinata.cloud/ipfs/{ipfs_hash}"

                # Update URL on-chain
                url_tx = deployed_contract.functions.updatePaymentUrl(
                    user_address,
                    index,
                    image_url
                ).build_transaction({
                    'from': account.address,
                    'gas': gas_limit,
                    'nonce': w3.eth.get_transaction_count(account.address),
                    'gasPrice': math.floor(w3.eth.gas_price * 1.2),
                    'chainId': chainId
                })
                
                signed_tx = w3.eth.account.sign_transaction(url_tx, private_key)
                tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
                tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

                updatePaymentStatus = deployed_contract.functions.updatePaymentStatus(user_address, index, 1).build_transaction({
                    'from': account.address,
                    'gas': gas_limit,
                    'nonce': w3.eth.get_transaction_count(account.address),
                    'gasPrice': math.floor(w3.eth.gas_price*1.2),
                    'chainId': chainId,
                })
                # Sign and send update transaction
                signed_tx = w3.eth.account.sign_transaction(updatePaymentStatus, private_key)
                raw_tx = getattr(signed_tx, 'rawTransaction', None) or getattr(signed_tx, 'raw_transaction')
                tx_hash = w3.eth.send_raw_transaction(raw_tx)
                tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
                print(f"Update transaction hash: {tx_hash.hex()}")
                print(f"Update transaction receipt: {tx_receipt}")
                print("Payment status updated successfully.")
            else:
                print("Nothing to process")
            
        except Exception as e:
            print(f"Error processing payment {index} for user {user_address}: {e}")

# Get all users who have made payments
def process_all_users():
    # Get user count
    user_count = deployed_contract.functions.getUserCount().call({'from': account.address})
    print(f"Total users: {user_count}")
    
    # Process each user's payments
    for i in range(user_count):
        try:
            user_address = deployed_contract.functions.allUsers(i).call({'from': account.address})
            # print(f"\nProcessing user {i + 1}/{user_count}: {user_address}")
            process_user_payments(user_address)
        except Exception as e:
            print(f"Error processing user {i}: {e}")
        

while True:
    # Run the processing
    process_all_users()
    sleep(5)

Total users: 1
Nothing to process
Nothing to process


KeyboardInterrupt: 

## Withdraw funds with the following cells

In [ ]:
# Check how many balance in the contract

prior = w3.from_wei(w3.eth.get_balance(exploit_address),'ether')
print(f'balance={prior}')


balance=20


In [ ]:
# (Owner only) to withdraw funds from the contract
prior = w3.from_wei(w3.eth.get_balance(account.address),'ether')
print(f'balance={prior}')

# Build transaction
tx = deployed_contract.functions.withdraw(w3.eth.get_balance(exploit_address)).build_transaction({
    'from': account.address,
    'gas': 8000000,  # Adjust gas as needed
    'gasPrice': w3.eth.gas_price,
    'nonce': w3.eth.get_transaction_count(account.address),
    'chainId': 9090  # Use your chain ID
})

# Sign and send transaction
signed_tx = w3.eth.account.sign_transaction(tx, private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

# Wait for transaction receipt
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(receipt['status'])

prior = w3.from_wei(w3.eth.get_balance(account.address),'ether')
print(f'balance={prior}')


balance=2501.67181792916824
1
balance=2521.67151355916824


In [45]:
receipt

AttributeDict({'blockHash': HexBytes('0x70d0c70eeb241763c10f02552e4d4a1c1faf1785b2b4f458ed7e8cd919ee3862'),
 'blockNumber': 391661,
 'contractAddress': None,
 'cumulativeGasUsed': 30565,
 'effectiveGasPrice': 10000000000,
 'from': '0xDc497AfB3203408EeCBED87E836F3587d882b522',
 'gasUsed': 30565,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': '0xb910CF7518DA19153B26a4aB25c7f9737bdEf48d',
 'transactionHash': HexBytes('0xec3a4a523bf03ccc74e76561c8a804afb087c9bbe4e045806a

In [23]:
private_key

'c9198de2ee0575e85cf34eb7853bfa3365242d152a865f4216c6b2f3c9847c47'

In [41]:
w3.eth.get_balance(exploit_address)

1

In [10]:
import requests
res = requests.get('https://meme-api.com/gimme/cryptocurrencymemes')
# Download the meme
url = res.json()["url"]
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
# urllib.request.urlopen(urllib.request.Request(meme_url, headers={"User-Agent": user_agent}))

# Make request to API
response = requests.get(url, stream=True)
response.raise_for_status()


In [9]:
res.json()

{'postLink': 'https://redd.it/1jrfkp2',
 'subreddit': 'cryptocurrencymemes',
 'title': 'INVESTMENTGONE',
 'url': 'https://i.redd.it/1t0s76u7duse1.png',
 'nsfw': False,
 'spoiler': False,
 'author': 'Plastic_Rough_5780',
 'ups': 22,
 'preview': ['https://preview.redd.it/1t0s76u7duse1.png?width=108&crop=smart&auto=webp&s=d12b6ca583589c09c37f0c826d6da0ca9deb34c1',
  'https://preview.redd.it/1t0s76u7duse1.png?width=216&crop=smart&auto=webp&s=4536610ff2a6ccb7273f2cb5f97460251c4c5808',
  'https://preview.redd.it/1t0s76u7duse1.png?width=320&crop=smart&auto=webp&s=cc0f8c6b7a6b6ed22ffb7528678997004f589c6e',
  'https://preview.redd.it/1t0s76u7duse1.png?width=640&crop=smart&auto=webp&s=e7777072198a76fd0a6c049d6b9efbb513797319']}